<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TensorBoard---First-Run" data-toc-modified-id="TensorBoard---First-Run-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>TensorBoard - First Run</a></span><ul class="toc-item"><li><span><a href="#He-initializer-model" data-toc-modified-id="He-initializer-model-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>He initializer model</a></span></li><li><span><a href="#ELU-model" data-toc-modified-id="ELU-model-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>ELU model</a></span></li><li><span><a href="#Result" data-toc-modified-id="Result-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Result</a></span></li></ul></li><li><span><a href="#TensorBoard---Second-Run" data-toc-modified-id="TensorBoard---Second-Run-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>TensorBoard - Second Run</a></span><ul class="toc-item"><li><span><a href="#Result" data-toc-modified-id="Result-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Result</a></span></li></ul></li><li><span><a href="#TensorBoard---Third-Run" data-toc-modified-id="TensorBoard---Third-Run-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>TensorBoard - Third Run</a></span><ul class="toc-item"><li><span><a href="#Result" data-toc-modified-id="Result-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Result</a></span></li></ul></li><li><span><a href="#TensorBoard---Fourth-Run" data-toc-modified-id="TensorBoard---Fourth-Run-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>TensorBoard - Fourth Run</a></span><ul class="toc-item"><li><span><a href="#Result" data-toc-modified-id="Result-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Result</a></span></li></ul></li></ul></div>

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from functools import partial

In [6]:
# Setup vars for the MINST data set
inputs = 28 * 28    # image dim in pixels
hidden1 = 300
hidden2 = 100
hidden3 = 50
hidden4 = 50
hidden5 = 50
outputs = 10

lr = 0.01
epochs = 20
batchSize = 50

mninst = input_data.read_data_sets("./datasets/mnist")

Extracting ./datasets/mnist\train-images-idx3-ubyte.gz
Extracting ./datasets/mnist\train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist\t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist\t10k-labels-idx1-ubyte.gz


In [7]:
def resetGraph(seed= 10):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [8]:
def cleanLogs():
    os.system('rm -rf ./logs/dnnTensorBoard/')

# TensorBoard - First Run

Initial run of the TensorBoard (TB).  Execute two models, and capture their loss and accuracy in two seperate plots.

## He initializer model

In [71]:
# Reset the TF CG
resetGraph()

# Clean away any old log files
cleanLogs()

# Set the TB logdir
logDir = './logs/dnnTensorBoard/heInit/train'

# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("NN"):
    # He initializer
    heInit = tf.contrib.layers.variance_scaling_initializer()
    
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.relu, kernel_initializer = heInit)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.relu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    # Capture loss
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("train"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Capture accuracy
    tf.summary.scalar("accuracy", accuracy)

init = tf.global_variables_initializer()

In [72]:
# Execute the TF CG
counter = 0

with tf.Session() as sess:
    init.run()
    
    # Create the TB writer and init
    trainWriter = tf.summary.FileWriter(logDir, sess.graph)
    merge = tf.summary.merge_all()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):      
            counter += 1       
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            summary, _ = sess.run([merge, opt], feed_dict = {x: xBatch, y: yBatch})
            
            # Capture summary data every N steps
            if counter % 500 == 0:
                merge = tf.summary.merge_all()
                trainWriter.add_summary(summary, counter)
        
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.9 Test Acc:  0.9039
10 Train Acc:  0.98 Test Acc:  0.9598
 
FINAL ::  Train Acc:  1.0 Test Acc:  0.9707


## ELU model

In [73]:
# Reset the TF CG
resetGraph()

# Set the TB logdir
logDir = './logs/dnnTensorBoard/elu/train'

# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("NN"):
    # Use ELU
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.elu)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.elu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    # Capture loss
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("train"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Capture accuracy
    tf.summary.scalar("accuracy", accuracy)

init = tf.global_variables_initializer()

In [74]:
# Execute the TF CG
counter = 0

with tf.Session() as sess:
    init.run()
    
    # Create the TB writer and init
    trainWriter = tf.summary.FileWriter(logDir, sess.graph)
    merge = tf.summary.merge_all()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):      
            counter += 1       
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            summary, _ = sess.run([merge, opt], feed_dict = {x: xBatch, y: yBatch})
            
            # Capture summary data every N steps
            if counter % 500 == 0:
                merge = tf.summary.merge_all()
                trainWriter.add_summary(summary, counter)
        
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.94 Test Acc:  0.9004
10 Train Acc:  1.0 Test Acc:  0.9471
 
FINAL ::  Train Acc:  1.0 Test Acc:  0.9612


## Result

![TB First Run](./images/tb-1st-run.png)


# TensorBoard - Second Run

Execute a single model, and capture loss and accuracy on the same plot.

In [11]:
# Reset the TF CG
resetGraph()

# Clean away any old log files
cleanLogs()

# Set the TB logdir - We want two log dirs since we are going to be plotting two values on the same plot
logDirTrain = './logs/dnnTensorBoard/secondRun/train'
logDirTest = './logs/dnnTensorBoard/secondRun/test'

# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("NN"):
    # He initializer
    heInit = tf.contrib.layers.variance_scaling_initializer()
    
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.relu, kernel_initializer = heInit)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.relu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    # Capture loss
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("train"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Capture accuracy
    tf.summary.scalar("accuracy", accuracy)

init = tf.global_variables_initializer()

# Execute the TF CG
counter = 0

with tf.Session() as sess:
    init.run()
    
    # Oddly enough if we want to write to values into the same plot we need TWO summary writers
    # One for training and one for test
    # Also notice we don't pass the 'sess.graph' arg to the 2nd summary writer instance
    # Also notice we don't call 'merge = tf.summary.merge_all()' other than this initial time
    # We add the summary values manually via calls to 'train_writer.add_summary(summaryObj, counterValue)'
    trainWriter = tf.summary.FileWriter(logDirTrain, sess.graph)
    testWriter = tf.summary.FileWriter(logDirTest)
    merge = tf.summary.merge_all()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):      
            counter += 1       
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            summary, _ = sess.run([merge, opt], feed_dict = {x: xBatch, y: yBatch})
            
            # Capture summary data every N steps
            if counter % 500 == 0:
                # Manually add the test accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: mninst.test.images, y: mninst.test.labels})
                testWriter.add_summary(summary, counter)
                
                # Manually add the train accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: xBatch, y: yBatch})
                trainWriter.add_summary(summary, counter)
                
        
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.96 Test Acc:  0.9021
10 Train Acc:  0.96 Test Acc:  0.9593
 
FINAL ::  Train Acc:  1.0 Test Acc:  0.97


## Result

![TB First Run](./images/tb-2nd-run.png)

# TensorBoard - Third Run

Execute a single model; capture loss and accuracy on the same plot; and capture metadata such as memory consumption.

In [11]:
# Reset the TF CG
resetGraph()

# Clean away any old log files
cleanLogs()

# Set the TB logdir - We want two log dirs since we are going to be plotting two values on the same plot
logDirTrain = './logs/dnnTensorBoard/thirdRun/train'
logDirTest = './logs/dnnTensorBoard/thirdRun/test'

# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("NN"):
    # He initializer
    heInit = tf.contrib.layers.variance_scaling_initializer()
    
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.relu, kernel_initializer = heInit)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.relu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    # Capture loss
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("train"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Capture accuracy
    tf.summary.scalar("accuracy", accuracy)

init = tf.global_variables_initializer()

# Execute the TF CG
counter = 0

with tf.Session() as sess:
    init.run()
    
    # Oddly enough if we want to write to values into the same plot we need TWO summary writers
    # One for training and one for test
    # Also notice we don't pass the 'sess.graph' arg to the 2nd summary writer instance
    # Also notice we don't call 'merge = tf.summary.merge_all()' other than this initial time
    # We add the summary values manually via calls to 'train_writer.add_summary(summaryObj, counterValue)'
    trainWriter = tf.summary.FileWriter(logDirTrain, sess.graph)
    testWriter = tf.summary.FileWriter(logDirTest)
    merge = tf.summary.merge_all()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):      
            
            counter += 1       
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            
            # Capture summary data every N steps
            if counter % 500 == 0:
                # Generate and capture metadata
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run(
                    [merge, opt], 
                    feed_dict = {x: xBatch, y: yBatch},
                    options=run_options,
                    run_metadata=run_metadata
                )
                
                # Write metadata
                trainWriter.add_run_metadata(run_metadata, 'step%03d' % counter)
                
                # Manually add the test accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: mninst.test.images, y: mninst.test.labels})
                testWriter.add_summary(summary, counter)
                
                # Manually add the train accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: xBatch, y: yBatch})
                trainWriter.add_summary(summary, counter)
            else:
                # Train as normal
                summary, _ = sess.run([merge, opt], feed_dict = {x: xBatch, y: yBatch})
                
        
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.9 Test Acc:  0.903
10 Train Acc:  1.0 Test Acc:  0.9594
 
FINAL ::  Train Acc:  0.98 Test Acc:  0.9701


## Result

![TB Third Run](./images/tb-3rd-run.png)

# TensorBoard - Fourth Run

Execute a single model multiple times with various hyperparameters; capture loss and accuracy on the same plot; and capture metadata such as memory consumption.

In [12]:
# Hyperparams
lr = 0.01
epochs = 20
batchSize = 50

# Reset the TF CG
resetGraph()

# Clean away any old log files
cleanLogs()

# Set the TB logdir - We want two log dirs since we are going to be plotting two values on the same plot
logDirTrain = './logs/dnnTensorBoard/fourthRun/1/train'
logDirTest = './logs/dnnTensorBoard/fourthRun/1/test'

# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("NN"):
    # He initializer
    heInit = tf.contrib.layers.variance_scaling_initializer()
    
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.relu, kernel_initializer = heInit)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.relu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    # Capture loss
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("train"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Capture accuracy
    tf.summary.scalar("accuracy", accuracy)

init = tf.global_variables_initializer()

# Execute the TF CG
counter = 0

with tf.Session() as sess:
    init.run()
    
    # Oddly enough if we want to write to values into the same plot we need TWO summary writers
    # One for training and one for test
    # Also notice we don't pass the 'sess.graph' arg to the 2nd summary writer instance
    # Also notice we don't call 'merge = tf.summary.merge_all()' other than this initial time
    # We add the summary values manually via calls to 'train_writer.add_summary(summaryObj, counterValue)'
    trainWriter = tf.summary.FileWriter(logDirTrain, sess.graph)
    testWriter = tf.summary.FileWriter(logDirTest)
    merge = tf.summary.merge_all()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):      
            
            counter += 1       
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            
            # Capture summary data every N steps
            if counter % 500 == 0:
                # Generate and capture metadata
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run(
                    [merge, opt], 
                    feed_dict = {x: xBatch, y: yBatch},
                    options=run_options,
                    run_metadata=run_metadata
                )
                
                # Write metadata
                trainWriter.add_run_metadata(run_metadata, 'step%03d' % counter)
                
                # Manually add the test accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: mninst.test.images, y: mninst.test.labels})
                testWriter.add_summary(summary, counter)
                
                # Manually add the train accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: xBatch, y: yBatch})
                trainWriter.add_summary(summary, counter)
            else:
                # Train as normal
                summary, _ = sess.run([merge, opt], feed_dict = {x: xBatch, y: yBatch})
                
        
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.88 Test Acc:  0.905
10 Train Acc:  0.96 Test Acc:  0.9598
 
FINAL ::  Train Acc:  0.94 Test Acc:  0.9709


In [13]:
# Alter hyperparams for second run
lr = 0.001
epochs = 30
batchSize = 100

# Reset the TF CG
resetGraph()

# Set the TB logdir - We want two log dirs since we are going to be plotting two values on the same plot
logDirTrain = './logs/dnnTensorBoard/fourthRun/2/train'
logDirTest = './logs/dnnTensorBoard/fourthRun/2/test'

# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("NN"):
    # He initializer
    heInit = tf.contrib.layers.variance_scaling_initializer()
    
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.relu, kernel_initializer = heInit)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.relu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    # Capture loss
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("train"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Capture accuracy
    tf.summary.scalar("accuracy", accuracy)

init = tf.global_variables_initializer()

# Execute the TF CG
counter = 0

with tf.Session() as sess:
    init.run()
    
    # Oddly enough if we want to write to values into the same plot we need TWO summary writers
    # One for training and one for test
    # Also notice we don't pass the 'sess.graph' arg to the 2nd summary writer instance
    # Also notice we don't call 'merge = tf.summary.merge_all()' other than this initial time
    # We add the summary values manually via calls to 'train_writer.add_summary(summaryObj, counterValue)'
    trainWriter = tf.summary.FileWriter(logDirTrain, sess.graph)
    testWriter = tf.summary.FileWriter(logDirTest)
    merge = tf.summary.merge_all()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):      
            
            counter += 1       
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            
            # Capture summary data every N steps
            if counter % 500 == 0:
                # Generate and capture metadata
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run(
                    [merge, opt], 
                    feed_dict = {x: xBatch, y: yBatch},
                    options=run_options,
                    run_metadata=run_metadata
                )
                
                # Write metadata
                trainWriter.add_run_metadata(run_metadata, 'step%03d' % counter)
                
                # Manually add the test accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: mninst.test.images, y: mninst.test.labels})
                testWriter.add_summary(summary, counter)
                
                # Manually add the train accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: xBatch, y: yBatch})
                trainWriter.add_summary(summary, counter)
            else:
                # Train as normal
                summary, _ = sess.run([merge, opt], feed_dict = {x: xBatch, y: yBatch})
                
        
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.53 Test Acc:  0.552
10 Train Acc:  0.91 Test Acc:  0.8856
20 Train Acc:  0.94 Test Acc:  0.9062
 
FINAL ::  Train Acc:  0.91 Test Acc:  0.9156


## Result

![TB Fourth Run](./images/tb-4th-run.png)